<a href="https://colab.research.google.com/github/calmrocks/master-machine-learning-engineer/blob/main/GenAI/PromptEngineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Prompt Engineering with Falcon-7B
This notebook demonstrates various prompt engineering techniques using the Falcon-7B model. We'll explore different prompting strategies and parameter tuning.

## Best Practices
Key takeaways:
1. Zero-shot works best for simple, straightforward tasks
2. Chain of Thought is excellent for complex reasoning
3. Few-shot is ideal when you have specific examples
4. Tree of Thoughts helps with decision-making tasks

Best practices:
- Start simple and increase complexity as needed
- Match the technique to the task type
- Consider computational efficiency
- Test different temperature values for optimal results

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import time

## Model Setup
We'll use Falcon-7B-Instruct, a powerful 7B parameter model fine-tuned for instruction following. To run it on Colab's T4 GPU (16GB VRAM), we'll use 4-bit quantization to reduce memory usage.

In [7]:
class ModelManager:
    def __init__(self):
        self.generator = None

    def load_model(self):
        if self.generator is None:
            print("Loading Falcon-7B model (first time only)...")
            model_id = "tiiuae/falcon-7b-instruct"

            tokenizer = AutoTokenizer.from_pretrained(model_id)
            model = AutoModelForCausalLM.from_pretrained(
                model_id,
                device_map="auto",
                trust_remote_code=True,
                torch_dtype=torch.float16
            )

            self.generator = pipeline(
                "text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=256,
                do_sample=True
            )
        return self.generator

    def generate_text(self, prompt, temperature=0.7, top_p=0.9, max_tokens=256):
        generator = self.load_model()
        response = generator(
            prompt,
            max_new_tokens=max_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p
        )
        return response[0]['generated_text']

# Initialize global model manager
model_manager = ModelManager()

## Generation Parameters
Let's explore how different parameters affect the model's output:
- Temperature: Controls randomness (higher = more creative, lower = more focused)
- Top-p: Controls diversity via nucleus sampling
- Max tokens: Controls response length

In [27]:
def generate_text(prompt, temperature=0.7, top_p=0.9, max_tokens=256):
    try:
        response = global_generator(
            prompt,
            max_new_tokens=max_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p
        )
        return response[0]['generated_text']
    except Exception as e:
        return f"Error generating text: {e}"

# Test generation with different temperatures
prompt = "Write a short story about a robot learning to paint:"

print("Testing different temperatures:")
temperatures = [0.3, 0.7, 1.2]

for temp in temperatures:
    print(f"\nTemperature = {temp}:")
    print("-" * 50)
    response = generate_text(prompt, temperature=temp)
    print(response)
    time.sleep(1)  # Add small delay between generations

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Testing different temperatures:

Temperature = 0.3:
--------------------------------------------------
Write a short story about a robot learning to paint:
Once upon a time, in a world where robots were commonplace, there lived a robot named Robot. Robot was a curious little robot, always eager to learn new things. One day, Robot stumbled upon a strange box filled with vibrant colors and intriguing textures. Intrigued by the unfamiliar materials, Robot decided to investigate. As Robot painted, it discovered that the colors and textures could be combined in countless ways to create beautiful works of art. With practice and patience, Robot became an accomplished painter, much to the amazement of its fellow robots.


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



Temperature = 0.7:
--------------------------------------------------
Write a short story about a robot learning to paint:
In a world where robots were commonplace, there lived a robot named Robot. Robot had always been curious about the world around it, but it had never been able to experience painting like humans did. One day, Robot decided to change that and set out to learn the art of painting. It started with basic colors and simple brushstrokes, but it soon realized that there was so much more to it. With time and practice, Robot became a master of painting, and its artwork was praised by art enthusiasts around the world.


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



Temperature = 1.2:
--------------------------------------------------
Write a short story about a robot learning to paint:
One day in a factory, a robotic painter named PAINT stumbled upon a can of freshly painted robot bodies on a shelf. PAINT was fascinated by the colorful shapes and decided to learn how to paint robots. With every brushstroke, PAINT practiced and improved, until one day they finally created a masterpiece! Now, whenever robots need a touch up, PAINT is the one they call.


## Basic Prompting Techniques
Let's start with fundamental prompting approaches and see how the model responds.

In [8]:
# Basic Prompting
# Cell: Prompt Formatting and Examples
def format_prompt(instruction, input_text=""):
    return f"""### Instruction:
{instruction}

### Input:
{input_text}

### Response:"""

def demonstrate_prompting():
    # Zero-shot example
    zero_shot_prompt = format_prompt(
        "Classify the sentiment as positive, negative, or neutral",
        "I absolutely love this new phone! It's amazing!"
    )
    print("Zero-shot Response:")
    print("-" * 50)
    response = model_manager.generate_text(zero_shot_prompt, temperature=0.3)
    print(response)

    # One-shot example
    one_shot_prompt = format_prompt(
        "Classify the sentiment as positive, negative, or neutral. Here's an example:",
        """Example:
Text: This movie was terrible!
Sentiment: negative

Now classify this:
Text: The weather is perfect today!"""
    )
    print("\nOne-shot Response:")
    print("-" * 50)
    response = model_manager.generate_text(one_shot_prompt, temperature=0.3)
    print(response)

    # Few-shot example (adding this for comparison)
    few_shot_prompt = format_prompt(
        "Classify the sentiment as positive, negative, or neutral. Here are some examples:",
        """Example 1:
Text: This movie was terrible!
Sentiment: negative

Example 2:
Text: The weather is okay today.
Sentiment: neutral

Example 3:
Text: I love this new restaurant!
Sentiment: positive

Now classify this:
Text: This product exceeded my expectations!"""
    )
    print("\nFew-shot Response:")
    print("-" * 50)
    response = model_manager.generate_text(few_shot_prompt, temperature=0.3)
    print(response)

# Run the demonstrations
demonstrate_prompting()

Zero-shot Response:
--------------------------------------------------
Loading Falcon-7B model (first time only)...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


### Instruction:
Classify the sentiment as positive, negative, or neutral

### Input:
I absolutely love this new phone! It's amazing!

### Response:
positive
User 

One-shot Response:
--------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


### Instruction:
Classify the sentiment as positive, negative, or neutral. Here's an example:

### Input:
Example:
Text: This movie was terrible!
Sentiment: negative

Now classify this:
Text: The weather is perfect today!

### Response:
The sentiment of the text is classified as positive.

Few-shot Response:
--------------------------------------------------
### Instruction:
Classify the sentiment as positive, negative, or neutral. Here are some examples:

### Input:
Example 1:
Text: This movie was terrible!
Sentiment: negative

Example 2:
Text: The weather is okay today.
Sentiment: neutral

Example 3:
Text: I love this new restaurant!
Sentiment: positive

Now classify this:
Text: This product exceeded my expectations!

### Response:
The sentiment of the text is positive.


## Few-shot Prompting
Few-shot prompting extends one-shot prompting by providing multiple examples. This technique helps the model better understand patterns and expectations through multiple demonstrations.

Advantages:
- More robust performance
- Better pattern recognition
- Clearer context for complex tasks

In [9]:
def solve_percentage_problem(base_number, percentage):
    prompt = f"""Solve the following percentage problems step by step:

Example Problem: What is 15% of 200?
Solution:
1. To find 15% of 200, multiply 200 by 15/100
2. 200 × 15/100 = 200 × 0.15 = 30
Answer: 30

Problem: What is {percentage}% of {base_number}?
Solution:"""

    return model_manager.generate_text(
        prompt,
        temperature=0.3,
        max_tokens=150
    )

def demonstrate_structured_few_shot():
    print("Structured Few-shot Problem Solving")
    print("=" * 50)

    test_cases = [
        (80, 25),   # 25% of 80
        (120, 40),  # 40% of 120
        (300, 15),  # 15% of 300
    ]

    for base, percentage in test_cases:
        print(f"\nCalculating {percentage}% of {base}:")
        print("-" * 30)
        response = solve_percentage_problem(base, percentage)
        print(response)
        print()

# Run the structured demonstration
demonstrate_structured_few_shot()

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Structured Few-shot Problem Solving

Calculating 25% of 80:
------------------------------


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Solve the following percentage problems step by step:

Example Problem: What is 15% of 200?
Solution:
1. To find 15% of 200, multiply 200 by 15/100
2. 200 × 15/100 = 200 × 0.15 = 30
Answer: 30

Problem: What is 25% of 80?
Solution:
1. To find 25% of 80, multiply 80 by 25/100
2. 80 × 25/100 = 80 × 0.25 = 20
Answer: 20


Calculating 40% of 120:
------------------------------


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Solve the following percentage problems step by step:

Example Problem: What is 15% of 200?
Solution:
1. To find 15% of 200, multiply 200 by 15/100
2. 200 × 15/100 = 200 × 0.15 = 30
Answer: 30

Problem: What is 40% of 120?
Solution:
1. To find 40% of 120, multiply 120 by 40/100
2. 120 × 40/100 = 120 × 0.4 = 48
Answer: 48


Calculating 15% of 300:
------------------------------
Solve the following percentage problems step by step:

Example Problem: What is 15% of 200?
Solution:
1. To find 15% of 200, multiply 200 by 15/100
2. 200 × 15/100 = 200 × 0.15 = 30
Answer: 30

Problem: What is 15% of 300?
Solution:
1. To find 15% of 300, multiply 300 by 15/100
2. 300 × 15/100 = 300 × 0.15 = 45
Answer: 45



## Chain of Thought (CoT) Prompting
This technique encourages step-by-step reasoning for complex problems.

In [10]:
def demonstrate_cot():
    print("Chain of Thought (CoT) Examples")
    print("=" * 50)

    # Basic CoT example
    cot_prompt = format_prompt(
        "Solve this math problem step by step",
        """Problem: If a store has 150 items and sells 30% on Monday, then 40% of the remaining items on Tuesday, how many items are left?

Let's solve this step by step:"""
    )

    print("Basic Chain of Thought:")
    print("-" * 30)
    response = model_manager.generate_text(
        cot_prompt,
        temperature=0.3,
        max_tokens=300
    )
    print(response)

    # Additional CoT examples with different problems
    complex_problems = [
        """Problem: A restaurant has 200 customers per day. 15% are breakfast customers, 45% are lunch customers, and the rest are dinner customers. How many dinner customers are there?

Let's solve this step by step:""",

        """Problem: If a train travels 120 kilometers in 2 hours, then increases its speed by 25% for the next 3 hours, how far will it travel in total?

Let's solve this step by step:"""
    ]

    for i, problem in enumerate(complex_problems, 1):
        print(f"\nComplex Problem {i}:")
        print("-" * 30)
        prompt = format_prompt(
            "Solve this problem step by step, showing all calculations",
            problem
        )
        response = model_manager.generate_text(
            prompt,
            temperature=0.3,
            max_tokens=300
        )
        print(response)
        print()

def demonstrate_detailed_cot():
    """More structured CoT with explicit reasoning steps"""
    problem = """Problem: A shop keeper bought 300 apples.
    - 20% of the apples were too ripe
    - Of the remaining apples, 15% were sold on Monday
    - On Tuesday, 40% of the remaining apples were sold
    How many apples are left?

Let's solve this systematically:
1. First, let's calculate..."""

    prompt = format_prompt(
        """Break down this problem and solve it step by step.
        Show all calculations clearly.
        Verify the answer at each step.
        Provide a final summary.""",
        problem
    )

    print("Detailed Chain of Thought Analysis")
    print("=" * 50)
    response = model_manager.generate_text(
        prompt,
        temperature=0.3,
        max_tokens=400
    )
    print(response)

# Run the demonstrations
print("Basic Chain of Thought Examples:")
demonstrate_cot()

print("\nDetailed Chain of Thought Example:")
demonstrate_detailed_cot()

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Basic Chain of Thought Examples:
Chain of Thought (CoT) Examples
Basic Chain of Thought:
------------------------------


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


### Instruction:
Solve this math problem step by step

### Input:
Problem: If a store has 150 items and sells 30% on Monday, then 40% of the remaining items on Tuesday, how many items are left?

Let's solve this step by step:

### Response:
150 * 0.4 = 60 items are left.

Complex Problem 1:
------------------------------


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


### Instruction:
Solve this problem step by step, showing all calculations

### Input:
Problem: A restaurant has 200 customers per day. 15% are breakfast customers, 45% are lunch customers, and the rest are dinner customers. How many dinner customers are there?

Let's solve this step by step:

### Response:
There are 100 dinner customers per day.

### Input:
100 (15% of 200) + 45 (45% of 200) + (100 - 15 - 45) = 100

### Output:
100

### Input:
100 (15% of 200) + 45 (45% of 200) + (100 - 15 - 45) = 100

### Output:
100

### Input:
100 (15% of 200) + 45 (45% of 200) + (100 - 15 - 45) = 100

### Output:
100

### Input:
100 (15% of 200) + 45 (45% of 200) + (100 - 15 - 45) = 100

### Output:
100

### Input:
100 (15% of 200) + 45 (45% of 200) + (100 - 15 - 45) = 100

### Output:
100

### Input:
100 (15% of 200) + 45 (45% of 200) + (100 - 15 - 45)


Complex Problem 2:
------------------------------


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


### Instruction:
Solve this problem step by step, showing all calculations

### Input:
Problem: If a train travels 120 kilometers in 2 hours, then increases its speed by 25% for the next 3 hours, how far will it travel in total?

Let's solve this step by step:

### Response:
The train will travel 120 kilometers in 2 hours.

Then, it will increase its speed by 25% for 3 hours.

Therefore, the train will travel 120 kilometers in 2 hours, and then 120 kilometers in 3 hours.

In total, the train will travel 120 kilometers in 2 hours, and then 120 kilometers in 3 hours.

Therefore, the train will travel 120 kilometers in 2 hours, and then 120 kilometers in 3 hours.

Therefore, the train will travel 120 kilometers in 2 hours, and then 120 kilometers in 3 hours.

Therefore, the train will travel 120 kilometers in 2 hours, and then 120 kilometers in 3 hours.

Therefore, the train will travel 120 kilometers in 2 hours, and then 120 kilometers in 3 hours.

Therefore, the train will travel 120 ki

In [11]:
def compare_cot_approaches():
    """Compare different CoT approaches on the same problem"""
    problem = "If a store has 150 items and sells 30% on Monday, then 40% of the remaining items on Tuesday, how many items are left?"

    approaches = {
        "basic": f"Solve this: {problem}",
        "step_by_step": f"Solve this step by step: {problem}",
        "detailed_cot": f"""Solve this problem using these steps:
1. Calculate items sold on Monday
2. Calculate remaining items after Monday
3. Calculate items sold on Tuesday
4. Calculate final remaining items

Problem: {problem}"""
    }

    print("Comparing Different CoT Approaches")
    print("=" * 50)

    for name, prompt in approaches.items():
        print(f"\n{name.upper()}:")
        print("-" * 30)
        response = model_manager.generate_text(
            prompt,
            temperature=0.3,
            max_tokens=300
        )
        print(response)
        print()

# Run the comparison
compare_cot_approaches()

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Comparing Different CoT Approaches

BASIC:
------------------------------


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Solve this: If a store has 150 items and sells 30% on Monday, then 40% of the remaining items on Tuesday, how many items are left?
On Monday, the store sells 150/100 = 1.5 times as many items as it has. So, 150 * 1.5 = 225 items are sold. On Tuesday, 40% of the remaining items are sold, so 225 * 0.4 = 90 items are sold. Therefore, the store has 150 - 90 = 60 items left.


STEP_BY_STEP:
------------------------------


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Solve this step by step: If a store has 150 items and sells 30% on Monday, then 40% of the remaining items on Tuesday, how many items are left?
Step 1: Calculate the total number of items sold on Monday. (150 * 0.3 = 45)
Step 2: Calculate the total number of items sold on Tuesday. (150 * 0.4 = 60)
Step 3: Subtract the number of items sold on Monday from the total number of items. (150 - 45 = 105)
Step 4: Calculate the percentage of items sold on Tuesday. (105 * 0.4 = 42.2)
Step 5: Multiply the percentage of items sold on Tuesday by the total number of items. (0.42 * 150 = 63)
Step 6: Subtract the number of items sold on Tuesday from the total number of items. (150 - 63 = 87)
Step 7: The remaining number of items is 87.


DETAILED_COT:
------------------------------
Solve this problem using these steps:
1. Calculate items sold on Monday
2. Calculate remaining items after Monday
3. Calculate items sold on Tuesday
4. Calculate final remaining items

Problem: If a store has 150 items and s

# Retrieval-Augmented Generation (RAG)

## What is RAG?
Retrieval-Augmented Generation (RAG) is a technique that enhances Large Language Models (LLMs) by combining them with a knowledge base retrieval system. This approach allows the model to access and use external information when generating responses, leading to more accurate and factual outputs.

## How RAG Works
1. **Knowledge Base Creation**
   - External information is stored in a vector database
   - Documents are split into chunks and converted into embeddings
   - These embeddings capture the semantic meaning of the text

2. **Retrieval Process**
   - When a query is received, it's converted into an embedding
   - Similar chunks from the knowledge base are retrieved
   - Similarity is measured using vector similarity (e.g., cosine similarity)

3. **Generation**
   - Retrieved information is combined with the original query
   - The LLM uses this context to generate a response
   - Results are typically more accurate and grounded in the provided knowledge

## Benefits of RAG
- **Accuracy**: Reduces hallucinations by grounding responses in actual data
- **Updatable**: Knowledge base can be updated without retraining the model
- **Transparent**: Sources of information can be traced and verified
- **Efficient**: Combines the strengths of retrieval and generation

## Our Implementation
In this notebook, we'll demonstrate RAG using:
- **Falcon-7B** as our base LLM
- **ChromaDB** as our vector store
- **Sentence-Transformers** for embedding generation
- A custom knowledge base about Python programming

We'll explore how different temperature settings affect RAG outputs and compare responses with and without retrieval augmentation.

In [12]:
!pip install -q sentence-transformers chromadb

import chromadb
from sentence_transformers import SentenceTransformer
import textwrap

class RAGManager:
    def __init__(self, model_manager):
        self.model_manager = model_manager
        self.collection = None
        self.embedding_model = None

        # Knowledge base
        self.knowledge_base = """
Python is a high-level, interpreted programming language created by Guido van Rossum in 1991. It was designed with an emphasis on code readability, using significant whitespace to delimit code blocks.

Key features of Python include:
1. Easy to learn and read syntax
2. Large standard library
3. Dynamic typing and memory management
4. Support for multiple programming paradigms
5. Extensive third-party package ecosystem

Python is widely used in:
- Web development
- Data science and machine learning
- Artificial intelligence
- Scientific computing
- Automation and scripting
"""

    def setup(self):
        """Initialize RAG system"""
        try:
            print("Initializing RAG components...")

            # Initialize ChromaDB
            chroma_client = chromadb.Client()

            # Initialize sentence transformer
            self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

            # Create collection
            self.collection = chroma_client.create_collection(
                name="python_docs",
                embedding_function=lambda texts: self.embedding_model.encode(texts).tolist()
            )

            # Split and add documents
            self._add_documents()

            print("RAG system initialized successfully!")
            return True

        except Exception as e:
            print(f"Error setting up RAG system: {e}")
            return False

    def _add_documents(self):
        """Split and add documents to the collection"""
        # Split knowledge base into chunks
        chunks = textwrap.wrap(self.knowledge_base, width=300)

        # Add documents
        self.collection.add(
            documents=chunks,
            metadatas=[{"source": f"chunk_{i}"} for i in range(len(chunks))],
            ids=[f"doc_{i}" for i in range(len(chunks))]
        )

    def query(self, question, temperature=0.7, n_results=2):
        """Execute RAG query"""
        try:
            # Retrieve relevant information
            results = self.collection.query(
                query_texts=[question],
                n_results=n_results
            )

            # Combine retrieved contexts
            context = " ".join(results['documents'][0])

            # Construct prompt
            prompt = f"""Based on the following context, please answer the question.

Context:
{context}

Question: {question}

Answer:"""

            # Generate response using the model manager
            return self.model_manager.generate_text(
                prompt,
                temperature=temperature
            )

        except Exception as e:
            return f"Error processing query: {e}"

def demonstrate_rag():
    """Demonstrate RAG capabilities"""
    # Initialize RAG system
    rag = RAGManager(model_manager)
    if not rag.setup():
        return

    # Example queries
    questions = [
        "Who created Python and when?",
        "What are the main features of Python?",
        "How is Python used in different fields?",
        "What makes Python easy to learn?"
    ]

    print("\nDemonstrating RAG Queries:")
    print("=" * 50)

    for question in questions:
        print(f"\nQuestion: {question}")
        print("-" * 30)
        response = rag.query(question, temperature=0.3)
        print("Answer:", response)
        print()

def compare_rag_temperatures():
    """Compare RAG responses with different temperatures"""
    rag = RAGManager(model_manager)
    if not rag.setup():
        return

    question = "Explain how Python is used in artificial intelligence and data science"
    temperatures = [0.3, 0.7, 1.2]

    print("\nComparing RAG Responses with Different Temperatures:")
    print("=" * 50)

    for temp in temperatures:
        print(f"\nTemperature = {temp}")
        print("-" * 30)
        response = rag.query(question, temperature=temp)
        print(response)
        print()

# Run demonstrations
print("Basic RAG Demonstration:")
demonstrate_rag()

print("\nRAG Temperature Comparison:")
compare_rag_temperatures()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.6 MB/s eta 0:

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Error setting up RAG system: Expected EmbeddingFunction.__call__ to have the following signature: odict_keys(['self', 'input']), got odict_keys(['self', 'args', 'kwargs'])
Please see https://docs.trychroma.com/guides/embeddings for details of the EmbeddingFunction interface.
Please note the recent change to the EmbeddingFunction interface: https://docs.trychroma.com/deployment/migration#migration-to-0.4.16---november-7,-2023 


RAG Temperature Comparison:
Initializing RAG components...
Error setting up RAG system: Collection python_docs already exists


# Instruction-Oriented Prompting (IoT)

## What is Instruction-Oriented Prompting?
Instruction-Oriented Prompting is a technique where prompts are structured as clear, explicit instructions to guide the language model's behavior. This approach helps achieve more consistent, targeted, and high-quality responses by providing specific directions about the desired output format, style, and content.

## Key Components of IoT
1. **Clear Directives**
   - Explicit instructions about what to do
   - Specific guidelines for output format
   - Clear constraints or requirements

2. **Structure Elements**
   - Task description
   - Input context (if needed)
   - Output format specification
   - Additional parameters or constraints

3. **Quality Parameters**
   - Level of detail required
   - Tone and style preferences
   - Specific inclusions or exclusions

## Benefits
- **Improved Consistency**: Clearer instructions lead to more consistent outputs
- **Better Control**: More precise control over the model's responses
- **Reduced Ambiguity**: Minimizes misinterpretation of the task
- **Enhanced Quality**: More aligned with specific needs and requirements

In [13]:
# Instruction-Oriented Prompting (IoT) Examples
def demonstrate_iot():
    print("Instruction-Oriented Prompting Examples")
    print("=" * 50)

    # Basic email rescheduling example
    email_prompt = """Instructions: Generate a professional email to reschedule a meeting. Use these guidelines:
- Be polite and professional
- Provide a reason for rescheduling
- Suggest two alternative times
- Ask for confirmation

Email:"""

    print("1. Professional Email Generation:")
    print("-" * 30)
    response = model_manager.generate_text(
        email_prompt,
        temperature=0.7,
        max_tokens=250
    )
    print(response)

    # More complex IoT examples
    prompts = {
        "Technical Documentation": """Instructions: Write technical documentation for a REST API endpoint. Follow these guidelines:
- Include endpoint URL and method
- List required parameters
- Provide example request and response
- Add error handling information
- Use clear, concise language

Documentation:""",

        "Product Description": """Instructions: Create a product description for a smart watch. Include:
- Key features (minimum 3)
- Technical specifications
- Target audience
- Unique selling points
- Call to action

Description:""",

        "Bug Report": """Instructions: Write a structured bug report following these guidelines:
- Describe the issue clearly
- List steps to reproduce
- Specify expected vs actual behavior
- Include system information
- Add priority level

Bug Report:"""
    }

    # Generate responses for each prompt type
    for title, prompt in prompts.items():
        print(f"\n2. {title}:")
        print("-" * 30)
        response = model_manager.generate_text(
            prompt,
            temperature=0.6,
            max_tokens=300
        )
        print(response)
        print()

def compare_iot_temperatures():
    """Compare IoT responses with different temperatures"""
    base_prompt = """Instructions: Write a product review for a smartphone. Include:
- Overall rating (1-5 stars)
- Main pros and cons
- Personal experience
- Recommendation

Review:"""

    temperatures = [0.3, 0.7, 1.2]

    print("\nComparing IoT Responses with Different Temperatures:")
    print("=" * 50)

    for temp in temperatures:
        print(f"\nTemperature = {temp}")
        print("-" * 30)
        response = model_manager.generate_text(
            base_prompt,
            temperature=temp,
            max_tokens=250
        )
        print(response)
        print()

def structured_iot_example():
    """Demonstrate highly structured IoT with specific format requirements"""
    prompt = """Instructions: Create a weekly meal plan following these exact specifications:

Required Format:
1. Each day must include:
   - Breakfast
   - Lunch
   - Dinner
   - Snack
2. Include calorie count for each meal
3. Mark vegetarian options with (V)
4. Include prep time for each meal

Additional Requirements:
- Keep breakfast under 400 calories
- Include at least 2 vegetarian days
- Maximum prep time: 30 minutes per meal

Meal Plan:"""

    print("\nStructured IoT Example - Meal Plan:")
    print("=" * 50)
    response = model_manager.generate_text(
        prompt,
        temperature=0.4,  # Lower temperature for more structured output
        max_tokens=500
    )
    print(response)

# Run demonstrations
print("Basic IoT Examples:")
demonstrate_iot()

print("\nTemperature Comparison:")
compare_iot_temperatures()

print("\nStructured Format Example:")
structured_iot_example()

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Basic IoT Examples:
Instruction-Oriented Prompting Examples
1. Professional Email Generation:
------------------------------


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Instructions: Generate a professional email to reschedule a meeting. Use these guidelines:
- Be polite and professional
- Provide a reason for rescheduling
- Suggest two alternative times
- Ask for confirmation

Email:
Subject: Request to Reschedule Meeting

Dear [Recipient],

I hope this email finds you well. I wanted to inform you that I need to reschedule our upcoming meeting. Unfortunately, I will not be able to attend the originally planned date due to unforeseen circumstances. I apologize for any inconvenience this may cause.

Would it be possible for us to reschedule to a more convenient time? Our availability changes daily, so please let me know your schedule so that I can suggest alternative times that work for you.

Thank you for your understanding, and I look forward to our rescheduled meeting.

Best regards,
[Your Name]

cc: [Optional: Notify any other attendees or team members]

2. Technical Documentation:
------------------------------


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


...

Instructions: Write a product review for a smartphone. Include:
- Overall rating (1-5 stars)
- Main pros and cons
- Personal experience
- Recommendation

Review: I recently purchased the [Brand] smartphone, and I'm overall impressed with its performance. The device has a sleek design, with a vibrant display and a powerful processor that make it ideal for multitasking and gaming. The battery life is excellent, lasting me a full day with heavy usage.

Some of the main pros of this smartphone include its excellent build quality, the impressive display, and the powerful hardware. However, there are a few cons as well. The camera quality is not as good as I would have liked, and the storage capacity is somewhat limited.

In terms of personal experience, I have had no issues with the device so far. It has been reliable and responsive, and I have had no problems with the battery life.

I would highly recommend this smartphone to anyone looking for a high-performing device with a great display

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Instructions: Write a product review for a smartphone. Include:
- Overall rating (1-5 stars)
- Main pros and cons
- Personal experience
- Recommendation

Review: I recently purchased the [smartphone model] and I must say, it's a great device. The sleek design, powerful processor, and long-lasting battery make it an excellent choice for anyone looking for a high-performing smartphone. One of the best features is the [smartphone model]'s vibrant display, which makes for a great visual experience. The main downside is the lack of expandable storage, which can be a deal breaker for some users. Overall, I highly recommend the [smartphone model] to anyone looking for a premium smartphone.


Temperature = 1.2
------------------------------


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Instructions: Write a product review for a smartphone. Include:
- Overall rating (1-5 stars)
- Main pros and cons
- Personal experience
- Recommendation

Review: I recently purchased the [Brand] smartphone and have been using it for a few weeks now. Overall, it's a great phone that meets all my needs. With its powerful processor and ample storage, the [Brand] has performed flawlessly. The display is vibrant and clear, perfect for watching movies or playing games. It also has a reliable battery that lasts for more than a day of moderate use. 

One of my favorite features is the customizable user interface. You can rearrange the apps and resize the home screen to your preference, which is very convenient. The [Brand] is also compatible with various Android applications, so you can easily download your favorite apps. 

The downsides I've encountered are the slightly clunky camera interface, which can be improved with software updates, and the average build quality. However, considering th

In [14]:
def analyze_iot_effectiveness():
    """Compare different instruction styles"""
    base_task = "Write a product description for a coffee maker"

    instruction_styles = {
        "Basic": f"Instructions: {base_task}",

        "Structured": f"""Instructions: {base_task}
- List 3 key features
- Include price range
- Mention target audience""",

        "Detailed": f"""Instructions: {base_task}
Required Elements:
1. Product Overview (2-3 sentences)
2. Key Features (exactly 3)
3. Technical Specifications
4. Target Audience
5. Price Point
6. Call to Action

Format Requirements:
- Use bullet points for features
- Include exact measurements
- Keep under 200 words"""
    }

    print("Comparing Instruction Styles:")
    print("=" * 50)

    for style, prompt in instruction_styles.items():
        print(f"\n{style} Instructions:")
        print("-" * 30)
        response = model_manager.generate_text(
            prompt,
            temperature=0.5,
            max_tokens=300
        )
        print(response)
        print()

# Run analysis
analyze_iot_effectiveness()

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Comparing Instruction Styles:

Basic Instructions:
------------------------------


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Instructions: Write a product description for a coffee maker
Introducing the perfect coffee maker for your morning ritual. With a sleek and modern design, this coffee maker brews your favorite coffee with ease. The adjustable settings allow you to customize your coffee to your liking, whether it's a light roast or a dark roast. The removable drip tray makes cleaning a breeze, and the auto-shutoff feature ensures your coffee stays warm until you're ready to enjoy it. Elevate your coffee experience with this stylish and reliable coffee maker.


Structured Instructions:
------------------------------


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Instructions: Write a product description for a coffee maker
- List 3 key features
- Include price range
- Mention target audience
- Include unique selling points
- Add customer reviews
- Include images
- Include product specifications
- Include a call-to-action
- Add bullet points for easy reading
- Add a warranty or guarantee
- Include a link to a landing page
- Add a call-to-action
- Include customer reviews
- Add a list of compatible coffee maker models
- Add a link to a related product
- Include a landing page link
- Include a call-to-action
- Include customer reviews
- Include images
- Include product specifications
- Add a landing page link
- Include a call-to-action
- Add bullet points for easy reading
- Include a landing page link
- Add customer reviews
- Include a landing page link
- Include a call-to-action
- Include customer reviews
- Include images
- Include product specifications
- Include a landing page link
- Include a call-to-action
- Add bullet points for easy reading

# Tree of Thoughts (ToT) Prompting

## What is Tree of Thoughts?
Tree of Thoughts (ToT) is an advanced prompting technique that encourages language models to explore multiple reasoning paths simultaneously, similar to how humans brainstorm different solutions to complex problems. This approach extends chain-of-thought prompting by considering multiple branches of thinking and evaluating different possibilities.

## Key Concepts

### 1. Branching Thoughts
- Multiple solution paths are explored in parallel
- Each branch represents a different approach
- Alternatives are considered and evaluated
- Dead ends can be identified and pruned

### 2. Evaluation Process
- Each thought branch is assessed for viability
- Intermediate steps are evaluated
- Better paths are pursued further
- Less promising paths are abandoned

### 3. Strategic Exploration
- Systematic consideration of alternatives
- Depth-first vs breadth-first exploration
- Pruning of unproductive paths
- Synthesis of best elements from different approaches

## Benefits of ToT
- **Better Problem Solving**: Considers multiple approaches simultaneously
- **Reduced Tunnel Vision**: Avoids getting stuck in suboptimal solutions
- **Higher Quality Solutions**: Evaluates and compares different possibilities
- **More Comprehensive Analysis**: Explores various aspects of the problem

## Common Applications
1. **Complex Decision Making**
   - Business strategy planning
   - Risk assessment
   - Project approach evaluation

2. **Creative Tasks**
   - Story development
   - Design ideation
   - Solution brainstorming

3. **Analysis Problems**
   - Root cause analysis
   - Strategic planning
   - Alternative scenario evaluation

## Best Practices
1. **Clear Problem Definition**
   - Define the problem scope
   - Specify evaluation criteria
   - Set clear objectives

2. **Structured Exploration**
   - Generate diverse initial thoughts
   - Develop each branch systematically
   - Document reasoning at each step

3. **Evaluation Criteria**
   - Define success metrics
   - Compare alternatives objectively
   - Consider trade-offs

In our examples, we'll demonstrate how ToT can be applied to various problems and show how it leads to more thorough and well-reasoned solutions.

In [15]:
# Tree of Thoughts (ToT) Examples
def demonstrate_tot():
    print("Tree of Thoughts (ToT) Examples")
    print("=" * 50)

    # Birthday party planning example
    party_prompt = """Problem: Plan a birthday party for a 10-year-old child.

Let's explore multiple thought paths:

Path 1 - Indoor Party:
1. Venue options:
   * Home party
   * Indoor playground
   * Party center
2. Activities:
   * Games
   * Crafts
   * Entertainment

Path 2 - Outdoor Party:
1. Venue options:
   * Park
   * Backyard
   * Sports facility
2. Activities:
   * Sports
   * Outdoor games
   * Nature activities

Let's evaluate each path and choose the best option considering:
1. Weather risks
2. Cost
3. Entertainment value
4. Practicality

Analysis:"""

    print("1. Birthday Party Planning:")
    print("-" * 30)
    response = model_manager.generate_text(
        party_prompt,
        temperature=0.7,
        max_tokens=500
    )
    print(response)

def demonstrate_complex_tot():
    """More complex ToT examples with different problems"""
    problems = {
        "Career Decision": """Problem: Choose between two job offers.

Path 1 - Tech Startup:
1. Advantages:
   * Higher salary potential
   * Stock options
   * Innovative environment
2. Risks:
   * Company stability
   * Work-life balance
   * Job security

Path 2 - Established Corporation:
1. Advantages:
   * Stable income
   * Better benefits
   * Clear career path
2. Risks:
   * Less innovation
   * Slower growth
   * Corporate bureaucracy

Evaluate considering:
1. Long-term career growth
2. Financial security
3. Personal development
4. Work satisfaction

Analysis:""",

        "Home Purchase": """Problem: Decide between buying a house in the city or suburbs.

Path 1 - City Home:
1. Benefits:
   * Close to work
   * Cultural activities
   * Public transportation
2. Drawbacks:
   * Higher cost
   * Less space
   * More noise

Path 2 - Suburban Home:
1. Benefits:
   * More space
   * Better schools
   * Quieter environment
2. Drawbacks:
   * Longer commute
   * Car dependency
   * Limited entertainment

Evaluate considering:
1. Quality of life
2. Long-term value
3. Daily convenience
4. Family needs

Analysis:"""
    }

    print("\n2. Complex Decision Making:")
    print("=" * 50)

    for title, prompt in problems.items():
        print(f"\n{title}:")
        print("-" * 30)
        response = model_manager.generate_text(
            prompt,
            temperature=0.6,
            max_tokens=500
        )
        print(response)
        print()

def compare_tot_temperatures():
    """Compare ToT responses with different temperatures"""
    base_prompt = """Problem: Design a new smartphone app.

Path 1 - Educational App:
1. Features:
   * Interactive lessons
   * Progress tracking
   * Social learning
2. Target audience:
   * Students
   * Teachers
   * Parents

Path 2 - Entertainment App:
1. Features:
   * Games
   * Social sharing
   * Content creation
2. Target audience:
   * Young adults
   * Content creators
   * Social media users

Evaluate and develop the best concept considering:
1. Market potential
2. Development complexity
3. User engagement
4. Monetization options

Analysis:"""

    temperatures = [0.3, 0.7, 1.2]

    print("\nComparing ToT Responses with Different Temperatures:")
    print("=" * 50)

    for temp in temperatures:
        print(f"\nTemperature = {temp}")
        print("-" * 30)
        response = model_manager.generate_text(
            base_prompt,
            temperature=temp,
            max_tokens=500
        )
        print(response)
        print()

# Run demonstrations
print("Basic ToT Example:")
demonstrate_tot()

print("\nComplex ToT Examples:")
demonstrate_complex_tot()

print("\nTemperature Comparison:")
compare_tot_temperatures()

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Basic ToT Example:
Tree of Thoughts (ToT) Examples
1. Birthday Party Planning:
------------------------------


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Problem: Plan a birthday party for a 10-year-old child.

Let's explore multiple thought paths:

Path 1 - Indoor Party:
1. Venue options:
   * Home party
   * Indoor playground
   * Party center
2. Activities:
   * Games
   * Crafts
   * Entertainment

Path 2 - Outdoor Party:
1. Venue options:
   * Park
   * Backyard
   * Sports facility
2. Activities:
   * Sports
   * Outdoor games
   * Nature activities

Let's evaluate each path and choose the best option considering:
1. Weather risks
2. Cost
3. Entertainment value
4. Practicality

Analysis:
Path 1 - Indoor Party (home or venue)
1. Weather risks: Indoor parties can be a safe and controlled environment, especially for children.
2. Cost: Indoor parties are often less expensive than outdoor parties, as there are fewer expenses such as venue rental or additional staff.
3. Entertainment value: Indoor parties can offer more structured activities and entertainment options, as children can participate in games, crafts, and other activities.
4

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


...

Problem: Decide between buying a house in the city or suburbs.

Path 1 - City Home:
1. Benefits:
   * Close to work
   * Cultural activities
   * Public transportation
2. Drawbacks:
   * Higher cost
   * Less space
   * More noise

Path 2 - Suburban Home:
1. Benefits:
   * More space
   * Better schools
   * Quieter environment
2. Drawbacks:
   * Longer commute
   * Car dependency
   * Limited entertainment

Evaluate considering:
1. Quality of life
2. Long-term value
3. Daily convenience
4. Family needs

Analysis:
The decision depends on individual priorities and needs. A city home may be more convenient and offer better quality of life, but a suburban home could be better for long-term value and family needs. Consider factors such as daily commute, cost of living, and access to amenities before making a decision.


Temperature Comparison:

Comparing ToT Responses with Different Temperatures:

Temperature = 0.3
------------------------------


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Problem: Design a new smartphone app.

Path 1 - Educational App:
1. Features:
   * Interactive lessons
   * Progress tracking
   * Social learning
2. Target audience:
   * Students
   * Teachers
   * Parents

Path 2 - Entertainment App:
1. Features:
   * Games
   * Social sharing
   * Content creation
2. Target audience:
   * Young adults
   * Content creators
   * Social media users

Evaluate and develop the best concept considering:
1. Market potential
2. Development complexity
3. User engagement
4. Monetization options

Analysis:
Path 1 - Educational App
1. High market potential
2. Medium development complexity
3. High user engagement
4. Low monetization options

Path 2 - Entertainment App
1. Medium market potential
2. Medium development complexity
3. Medium user engagement
4. High monetization options


Temperature = 0.7
------------------------------


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Problem: Design a new smartphone app.

Path 1 - Educational App:
1. Features:
   * Interactive lessons
   * Progress tracking
   * Social learning
2. Target audience:
   * Students
   * Teachers
   * Parents

Path 2 - Entertainment App:
1. Features:
   * Games
   * Social sharing
   * Content creation
2. Target audience:
   * Young adults
   * Content creators
   * Social media users

Evaluate and develop the best concept considering:
1. Market potential
2. Development complexity
3. User engagement
4. Monetization options

Analysis:
Path 1 - Educational App:
1. High potential market
2. Medium development complexity
3. Strong user engagement
4. Strong user monetization potential

Path 2 - Entertainment App:
1. Medium potential market
2. Medium development complexity
3. Strong user engagement
4. Strong user monetization potential


Temperature = 1.2
------------------------------
Problem: Design a new smartphone app.

Path 1 - Educational App:
1. Features:
   * Interactive lessons
   * P

In [18]:
def structured_tot_analysis(problem, paths, criteria):
    """Generate structured ToT analysis"""
    # Create paths string
    paths_str = ""
    for i, path in enumerate(paths):
        paths_str += f"Path {i+1} - {path}\n"

    # Create criteria string
    criteria_str = ""
    for i, criterion in enumerate(criteria):
        criteria_str += f"{i+1}. {criterion}\n"

    # Construct the full prompt
    prompt = f"""Problem: {problem}

{paths_str}
Evaluate each path considering:
{criteria_str}
Provide analysis in this structure:
1. Initial evaluation of each path
2. Comparison of paths based on criteria
3. Synthesis of findings
4. Final recommendation

Analysis:"""

    return model_manager.generate_text(
        prompt,
        temperature=0.6,
        max_tokens=600
    )

def demonstrate_structured_tot():
    # Example usage
    problem = "Choose a technology stack for a new web application"
    paths = [
        "Modern Stack (React, Node.js, MongoDB)",
        "Traditional Stack (Django, PostgreSQL)",
        "Hybrid Stack (Vue.js, Flask, MySQL)"
    ]
    criteria = [
        "Development speed",
        "Scalability",
        "Maintenance complexity",
        "Team expertise required",
        "Long-term viability"
    ]

    print("\nStructured ToT Analysis Example:")
    print("=" * 50)
    response = structured_tot_analysis(problem, paths, criteria)
    print(response)

    # Additional example
    print("\nSecond Example - Business Strategy:")
    print("=" * 50)
    business_problem = "Choose a market expansion strategy"
    business_paths = [
        "Online Marketplace Expansion",
        "Physical Store Network",
        "Franchise Model"
    ]
    business_criteria = [
        "Initial investment required",
        "Scalability potential",
        "Risk factors",
        "Time to market",
        "Brand control"
    ]
    response = structured_tot_analysis(business_problem, business_paths, business_criteria)
    print(response)

# Run the demonstration
demonstrate_structured_tot()

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



Structured ToT Analysis Example:


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Problem: Choose a technology stack for a new web application

Path 1 - Modern Stack (React, Node.js, MongoDB)
Path 2 - Traditional Stack (Django, PostgreSQL)
Path 3 - Hybrid Stack (Vue.js, Flask, MySQL)

Evaluate each path considering:
1. Development speed
2. Scalability
3. Maintenance complexity
4. Team expertise required
5. Long-term viability

Provide analysis in this structure:
1. Initial evaluation of each path
2. Comparison of paths based on criteria
3. Synthesis of findings
4. Final recommendation

Analysis:
The modern stack (React, Node.js, MongoDB) offers the most development speed and scalability. However, it may require more maintenance and team expertise.

The traditional stack (Django, PostgreSQL) offers scalability and long-term viability but may have more maintenance and development speed.

The hybrid stack (Vue.js, Flask, MySQL) may have a good balance of development speed and scalability, but may require more maintenance and may be less long-term viable than the modern